In [1]:
import pandas as pd
import numpy as np
import re
from itertools import combinations, combinations_with_replacement

#vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

#gridsearch
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score

#classifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

#words
from nltk.corpus import stopwords
from stop_words import get_stop_words

# Разработка сентимент-анализа под задачу заказчика
К вашей компании пришел заказчик, которому нужно решение задачи анализа тональности отзывов на товары. Заказчик хочет, чтобы вы оценили возможное качество работы такого алгоритма на небольшой тестовой выборке. При этом больше никаких данных вам не предоставляется. Требуется, чтобы качество работы вашего алгоритма (по accuracy) было строго больше 85%.

Оценка качества в этом задании реализована через контест на Kaggle Inclass:

* https://inclass.kaggle.com/c/product-reviews-sentiment-analysis

Вам предстоит посмотреть на предоставленные заказчиком отзывы, собрать похожие отзывы в качестве обучающей выборки, и поэкспериментировать с постановкой задачи (разметкой вашей выборки на позитивные и негативные примеры) так, чтобы результат на примерах заказчика был по возможности получше.

Обратите внимание, что заказчик предоставил всего 100 примеров в качестве тестовой выборки - ситуация, когда размеченных данных почти нет - вообще очень частая в индустриальном анализе данных. Конечно, эти отзывы можно было бы идеально разметить вручную и получить максимальное качество, но вы сами не заинтересованы в таком подходе, т.к. потом придется и на всех новых примерах демонстрировать заказчику идеальную работу, что, конечно, вряд ли будет по силам алгоритму. В любом случае рано или поздно алгоритм придется разрабатывать, поэтому попытки "сжульничать" и не делать никакой модели не одобряются.

Для оценки качества в этом соревновании будет использоваться accuracy.

## Данные клиента

In [2]:
with open("data/test.csv", encoding = 'utf-8') as f:
    raw_text = f.read()
df_test = pd.DataFrame(re.findall(r'<review>([\s\S]*?)</review>', raw_text),columns = ['text'])
df_test.values[:3]

array([['Ужасно слабый аккумулятор, это основной минус этого аппарата, разряжается буквально за пару часов при включенном wifi и на макс подсветке, например если играть или смотреть видео, следовательно использовать можно только если есть постоянная возможность подзарядиться. Качества звука через динамик далеко не на высоте.Наблюдаются незначительные тормоза в некоторых приложениях и вообще в меню. Очень мало встроенной памяти, а приложения устанавливаются именно туда, с этим связанны неудобства - нужно постоянно переносить их на карту памяти.\nНесколько неудобно что нету отдельной кнопки для фото. Подумываю купить батарею большей емкость мб что нибудь измениться.\n'],
       ['ценанадежность-неубиваемостьдолго держит батарею 4 дня стабильно как телефон, 3-4 как плеер если \nпостоянно долбиться в уши и звонить по паре часо на дню, игры и, конечно,  смс , в месяц около 200 шт набирается.\n Максимальное время работы 5 дней в щадящем режиме.2 simqwerty рулит -после нее набор смс на обычны

## Парсинг дополнительных данных
Отзывы посвящены мобильным телефонам. Данных мало, они не размечены. Попробуем собрать отзывы самим через парсинг сайтов отзывов с телефонами - https://www.e-katalog.ru/. При парсинге заодно будем смотреть на оценку в отзыве - на этом сайте за это отвечает смайл в заголовке отзыва:
* '/img/svg/review-smile-1.svg','/img/svg/review-smile-2.svg' - грустные смайлы, пользователь не доволен
* '/img/svg/review-smile-3.svg','/img/svg/review-smile-4.svg' - веселые смайлы, пользователь доволен

Чтобы парсить отзывы было быстрее - подключим многозадачность с помощью multiprocessing. Увы, библиотека не дружит с Jupyter - поэтому запустим parse_phone_reviews.py

In [3]:
%%writefile parse_phone_reviews.py
import requests
import bs4
from multiprocessing import Pool
from functools import reduce
import pandas as pd
import numpy as np

def getAllTags(url, tag, attrs):
    req = requests.get(url)
    parser = bs4.BeautifulSoup(req.text, 'lxml')
    return parser.findAll(tag, attrs)
    
def parse_urls(url):
    url_tags = getAllTags(url, 'a', 'model-short-title')
    urls = ['https://www.e-katalog.ru/review' + tag['href'][:-4] for tag in url_tags]
    return urls

def div_to_text(div):
    if div is None:
        return None
    else:
        return div.get_text(separator="\n") + '\n'

def parse_review(parser):
    description = div_to_text(parser.find('span', attrs={'itemprop':'description'}))
    title = div_to_text(parser.find('div', 'review-title'))
    plus = div_to_text(parser.find('div', 'review-plus'))
    minus = div_to_text(parser.find('div', 'review-minus'))
    result = ''
    if description is None:
        result = title + plus + minus
    else:
        result = title + description + plus + minus
    return result

def parse_sentiment(parser):
    tag = parser.find('div', 'review-title')
    if tag.find('img')['src'] in ['/img/svg/review-smile-3.svg','/img/svg/review-smile-4.svg']:
        return 1
    else:
        return 0

def parse_sentiment_reviews(url):
    tags = getAllTags(url, 'td', 'review-td')
    reviews_n_classes = []
    for tag in tags:
        review = parse_review(tag)
        class_ = parse_sentiment(tag)
        reviews_n_classes.append([review, class_])
    return reviews_n_classes

def map_n_reduce(function, array, processes=8):
    p = Pool(processes)
    map_ = p.map(function, array)
    return reduce(lambda x,y: x+y, map_)    

if __name__ == '__main__':    
    main_urls = ['https://www.e-katalog.ru/list/122/'] + ['https://www.e-katalog.ru/list/122/' + str(i) for i in np.arange(1,76 + 1)]
    reduce_urls = map_n_reduce(parse_urls, main_urls)
    reviews = map_n_reduce(parse_sentiment_reviews, reduce_urls)
    pd.DataFrame(reviews, columns=['text','class']).to_csv('data/train.csv', index=False)

Writing parse_phone_reviews.py


In [4]:
df_train = pd.read_csv('data/train.csv')
df_train.values[:3]

array([['Вполне хороший смарт, достаточно шустрый  камера хорошая, зарядка быстрая\nПокупайте на 128 /6 гб!!! камера жрет много памяти!!! 64 не думаю что хватит, покупайте ещё чехол!!! Зад телефона стеклянный - то есть легко разобьется, и да фирменный чехол внутри был и он давит на кнопки выключения и звука, не пожалеете если еще 2-3к сверху скините, покупал за 17 990р - 128/6 белый цвет\r\nПользуюсь неделю, все пока что идеально! ;)\nШустрый - тянет многие игры, я сам не особо играю изредка захожу в Pubg, Clash of clans и др... Играя держит стабильно 57-60 fps - зарядка 5-6 часов игры держит \r\nПокупал за 17 990 - 128/6 гб ,  в  М.видио\r\nМногие писали про нагрев, я не ощущал ничего подобного, Фризов не было ни разу, батарея держит 1-2 дня (при моем использовании) для простого использования с головы хватит) памяти много и то здорово) Камера 64мп снимает фотографии 15-22мб это не мало) снимает ни хорошо и не плохо - где-то золотая середина, макро съемка прям супер)) телефон заряжаетс

In [5]:
review_pos_count = df_train['class'].sum()
review_count = len(df_train)
print('Количество отзывов в обучающей выборке', review_count)
print('Позитивных отзывов больше. Cоотношение классов', review_pos_count/review_count)

Количество отзывов в обучающей выборке 4146
Позитивных отзывов больше. Cоотношение классов 0.7496382054992764


### Обучение на распарсенных данных

In [6]:
comb = np.arange(1,11)
ngram_range = list(combinations_with_replacement(comb, 2))[:10]
param_grid = {
    'vect': [TfidfVectorizer()],
    'vect__ngram_range': ngram_range,
    'vect__stop_words': ['russian', stopwords.words('russian'), get_stop_words('ru'), None],
    'clf':[LinearSVC(),LogisticRegression()],    
    'clf__C': np.arange(0.2, 2.1 ,0.2),
    'clf__class_weight': ['balanced', None]
}

In [7]:
pipe = Pipeline(steps=[('vect', TfidfVectorizer(ngram_range = [1,6])), ('clf', LinearSVC(C=1.0))])
search = GridSearchCV(pipe, param_grid, verbose = 1, scoring = 'accuracy', n_jobs=-1, cv = 5)
search = search.fit(df_train['text'], df_train['class'])

Fitting 5 folds for each of 1600 candidates, totalling 8000 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed: 14.9min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed: 22.0min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed: 30.4min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed: 40.3min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed: 51.2min
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed: 69.3min
[Parallel(n_jobs=-1)]: Done 6034 tasks      | elapsed: 92.9min
[Parallel(n_jobs=-1)]: Done 7184 tasks      | elapsed: 122.7min
[Parallel(n_jobs=-1)]: Done 8000 out of 8000 | elapsed: 147.4min finished


In [13]:
print(search.best_score_)
print(search.best_params_)

0.9119662243667068
{'clf': LogisticRegression(C=1.6, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False), 'clf__C': 1.6, 'clf__class_weight': 'balanced', 'vect': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None), 'vect__ngram_range': (1, 2), 'vect__stop_words': None}


## Оценка на тестовой выборке
Для удобства я заранее разметил вручную 100 отзывов от клиента. Так будет удобно сравнить ручную разметку и автоматическую на распарсенных данных.

P.S. В отрицательных отзывах чаще всего встречаются слова "не рекомендую к покупке", "полная хрень", "вы пожалеете", "НЕ ПОКУПАЙТЕ" и т.п. Даже по этим n-граммам можно уже делать анализ тональности)

In [14]:
df_test = pd.read_csv('data/test_class.csv')

In [15]:
df_test['class_pred'] = search.predict(df_test['text'])

In [16]:
df_test.head()

,text,class,class_pred
0,"Ужасно слабый аккумулятор, это основной минус ...",0,0
1,ценанадежность-неубиваемостьдолго держит батар...,1,1
2,"подробнее в комментариях\nК сожалению, факт по...",0,0
3,я любительница громкой музыки. Тише телефона у...,0,0
4,"Дата выпуска - 2011 г, емкость - 1430 mAh, тех...",1,1


In [17]:
score = accuracy_score(df_test['class'], df_test['class_pred'])
score_threshold = 0.85
print('Точность на тестовой выборке -', score)
print('Порог точности по ТЗ -', score_threshold)
print('Точность выше порога -', score > score_threshold)

Точность на тестовой выборке - 0.9
Порог точности по ТЗ - 0.85
Точность выше порога - True


In [23]:
df_test['y'] = df_test['class_pred'].map({0:'neg', 1:'pos'})
df_test.index.names = ['Id']
df_test[['y']].to_csv('results/pred.csv')

Результат хороший - учитывая, что обучающую выборку мы сделали сами на основе категории отзывов из тестовой. На kaggle получился результат 0.88000. Идеальной точности добиться на тестовой можно, но это будет нечестно, если загрузить размеченную самими тестовую выборку. В любом случае алгоритм будет немного ошибаться - 90% точности очень хороший результат, который работает как на тестовой, так и на обучающей. Зато без переообучения Результат хороший - учитывая, что обучающую выборку мы сделали сами на основе категории отзывов из тестовой. На kaggle получился результат 0.88000. Идеальной точности добиться на тестовой можно, но это будет нечестно, если загрузить размеченную самими тестовую выборку. В любом случае алгоритм будет немного ошибаться - 90% точности очень хороший результат, который работает как на тестовой, так и на обучающей. Зато без переообучения 🤯.